
#Constraint Satisfaction using SMT solvers

In this notebook, we will
look at SMT solvers and see how they can be used to formalize and solve constraint satisfaction problems. Particularly, we will show how to encode the Matching problem using constraints on a set of boolean variables.


**Instructions:**
1. To get started, click on File on the top left and click "Save a copy in Drive."
This will give you an editable version of this document that you can use.
2. If you press `CMD`+`Enter` it runs the cell, and if you press `Shift`+`Enter` it runs the cell and goes to the next one.
3. Make sure you run all cells as you go through the notebook; some cells will not work properly unless the previous one
has been run too.
4. If you disconnect or are inactive for some time you should run all of the cells again.

## 0. Preliminaries (you should run this cell but there is no need to read it)

In [ ]:
!pip install z3-solver
!pip install git+https://github.com/crrivero/FormalMethodsTasting.git#subdirectory=core
from z3 import *
from tofmcore import showSolver, draw_bipartite_graph, print_matching_solution
from IPython.display import clear_output
clear_output()

## Encoding constraints in Z3

The goal of this notebook is to teach you about formal methods;
particularly, how you can use existing formal verification tools
(in this case, Z3) to analyze and solve your own problems.
Before we get started, let's look at some basic things we can do with Z3.

### Boolean

Suppose you have the following three boolean constraints and you want to check if there's a solution (an assignment of the variables) that satisfies all of them:

$$ x_1 \lor x_2 \lor x_3 $$

$$ \neg x_1 \implies \neg x_2$$

$$  x_1 \land x_3  $$

Let's see how we can do this using Z3.

In [ ]:
s = Solver() # initialize Z3 solver

# initialize variables

x1 = Bool('x_1') # declaring that x_1 is a boolean variable in Z3 which will be referred to as x1 in Python
x2 = Bool('x_2')
x3 = Bool('x_3')

# Note: we can also initialize multiple variables like so: x1, x2, x3 = Bools('x_1 x_2 x_3')

# we use s.add(.) to add a constraint to our solver s
# constraints can be made using many different operations such as Or, And, Not,
# equality, etc.

# here's how we would add the constraints above to our solver:

s.add( Or( x1, x2, x3 ) ) # add the first constraint
s.add( Implies( Not(x1), Not(x2) ) ) # add the second constraint
s.add( And( x1, x3 ) ) # add the third constraint

In [ ]:
# to view the constraints in our solver, we can use the following:
print( s )
# this prints the constraints as they appear in Z3 using Z3's notation

For better readability, this notebook also has a custom print function to view our constraints in LaTeX format, like so:

In [ ]:
showSolver( s )

In [ ]:
# we can use s.check() to run the solver and check whether its satisfiable:
print ( s.check() )

 "sat" means our system of constraints is satisfiable

In [ ]:
# after using s.check(),  we can use s.model() to output a solution if one exsits
solution = s.model()
print( solution )

Let's modify our system of constraint a bit and see if it's still satisfiable. Suppose we want to check if there's a solution where $x_1 = \neg x_3$. Let's see how we would do this with Z3.

In [ ]:
s.add( x1 == Not(x3) )
showSolver( s )

In [ ]:
print( s.check() ) # check if solution exists with new constraint

"unsat" means the system is not satisfiable, i.e., there is no assignment on the variables $x_1$, $x_2$, and $x_3$ that satisfies all the constraints we gave to the solver. **Note that if we were to run s.model() now we would get an error.**

We can also add arithmetic constraints to boolean variables:

$$ x_1 + x_2 + x_3 = 1 $$

To model these types of constraints, Z3 converts True/False boolean values to 1/0 integer values.

In [ ]:
s = Solver() # initialize solver

# initialize variables
x1 = Bool('x_1')
x2 = Bool('x_2')
x3 = Bool('x_3')

s.add( x1 + x2 + x3 == 1 )

print( s )

Note how Z3 uses If conditions to convert boolean variables into integers.
Our custom printer function prints these without the if conditions for better readability.

In [ ]:
showSolver( s )

In [ ]:
print( s.check() ) # check if satisfiable
print( s.model() ) # print solution

Now it's your turn! Complete the code in the cell below to find a solution satisfying the following constraints:

$$ x_1 \lor x_2 \lor x_3 $$
$$ x_1 + x_2 = x_3 $$
$$ \neg x_1 \implies x_3 $$

In [ ]:
s = Solver() # initialize solver

# initialize variables
x1 = Bool('x_1')
x2 = Bool('x_2')
x3 = True # REPLACE THIS LINE

s.add( Or( x1, x2 ) ) # REPLACE THIS LINE
s.add( x1 + x2 == 1 ) # REPLACE THIS LINE
s.add( x2 == x2 )  # REPLACE THIS LINE

showSolver( s )
print( s.check() )
print( s.model() )

Note that Z3 does not support this boolean-to-integer conversion for every operator. For example, adding the constraint `x1 >= 1` or `x1 == 1` to our solver would give an error. Proper usage dictates using integers when using arithmetic constraints, but to keep things simple for our introductory notebook we will still work with booleans.

## Matching using Z3

Consider a set of employees and a set of tasks. Suppose that each employee chooses which task they are willing to do. We can model this relationship using the following graph, where left vertices represent employees and right vertices represent tasks. Edges represent employee preferences: an employee and task are joined by an edge if the employee is okay with doing the task.


In [ ]:
employees = [ 'Alice', 'Bob', 'Carl' ]
tasks = [ 'Print', 'Talk', 'Write' ]

taskPreferences = [
    ( 'Alice', 'Print' ), # Alice can perform the Print task
    ( 'Alice', 'Talk' ),
    ( 'Alice', 'Write' ),
    ( 'Bob', 'Talk' ),
    ( 'Carl', 'Print' ),
    ( 'Carl', 'Talk' )
]
draw_bipartite_graph( employees, tasks, taskPreferences )

We want to assign tasks to each employee so that each task is assigned to an employee, and each employee has at most one task assigned to them, i.e., we want to **match** employees and tasks such that each task is assigned to an employee.

Formally, a **matching** is a collection of edges where no two vertices share an edge. We want to find a matching where each vertex on the right is part of an edge in the matching.

Let's see how to do that using Z3.

<!-- For the sake of completeness, we now provide some formal definitions. A bipartite graph  -->
<!-- Formally, given two disjoint sets of vertices $V_1$ and $V_2$ in a graph, a **matching** is a set of edges $M$ such that: (1) no two edges in $M$ share a vertex, and (2) each edge in $M$ has one vertex in $V_1$ and one vertex in $V_2$. -->

#### Step 1: Make boolean variables for each edge

In our solution, an edge with the value of True will represent an edge that has been selected in our matching.

In [ ]:
# Alice shares an edge with Print, we will represent that edge with the variable AlicePrint:
AlicePrint = Bool( 'AlicePrint' )

# We follow the same naming convention for our other variables

AliceTalk = Bool( 'AliceTalk' )
AliceWrite = Bool( 'AliceWrite' )
BobTalk = Bool( 'BobTalk' )
CarlPrint = Bool( 'CarlPrint' )
CarlTalk = Bool( 'CarlTalk' )

#### Step 2: Initialize solver

In [ ]:
s = Solver()

#### Step 3: Write constraints

In [ ]:
# Each left vertex (Employee) can have at most one edge adjacent to it in the matching
s.add( AlicePrint + AliceTalk + AliceWrite <= 1 )
s.add( CarlPrint + CarlTalk <= 1 )
# Note that we do not have to add the constraint "Bob_Talk <= 1" since Bob_Talk
# is a boolean variable and its value is always at most 1 when converted to
# an integer

# Each right vertex (Task) can have at most one edge adjacent to it in the matching
s.add( AlicePrint + CarlPrint <= 1 )
s.add( AliceTalk + BobTalk + CarlTalk <= 1 )

# Now we need to add a constraint that ensures all of the right vertices are
# part of an edge in the matching. Since there are three tasks, we can do this
# by asking for the sum of all edges going into these tasks be at least three:

s.add( AlicePrint + CarlPrint + AliceTalk + BobTalk + CarlTalk + AliceWrite >= 3 )

# Let's see what our constraints look like
showSolver( s )

#### Step 4: Check if a solution exists

In [ ]:
print( s.check() )

####Step 5: View solution

In [ ]:
solution = s.model()
print(solution)

We have also defined a function to visualize our solution:

In [ ]:
print_matching_solution( solution )


### Congratulations! You just wrote an SMT solver to find matchings!


####If you'd like to continue your Z3 journey, you can start with this guide to learn more:
https://ericpony.github.io/z3py-tutorial/guide-examples.htm